In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [36]:

# Load input data
X = pd.read_csv('data/train/X_Train_Data_Input.csv')

# Load target data
Y = pd.read_csv('data/train/Y_Train_Data_Target.csv')

# Display the first few rows to verify the contents
X.head()
Y.head()


x = X.iloc[:,1:]
y = Y.iloc[:,1:]

In [37]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
x.head()
print("No.of rows : ",x.shape[0])
print("No.of Columns : ",x.shape[1])

In [ ]:
#checking each row is aligned correctly with corresponding id
are_ids_aligned = X['ID'].equals(Y['ID'])
if are_ids_aligned:
    print(x.describe())


In [ ]:

total = x.isnull().sum().sort_values(ascending=False)

percent = (x.isnull().sum()/x.isnull().count()).sort_values(ascending=False)

null_percent_table = pd.concat([total,percent],axis=1,keys=['Total','Percent'])

null_percent_table

In [ ]:
#applying imputation to generate values for null values using the algorithms Using LightGBM
xdata = x.copy()
xdata.isnull().sum().sort_values(ascending=False)

#importing lightGBM imputation function to impute the missing values
import algorithm_scripts.null_imputation as nm
xdata_imputed = nm.lightgbm_impute(xdata)
print(xdata_imputed.isnull().sum())

xdata_imputed.head(5)


In [ ]:
xdata_imputed.describe()

In [ ]:
#analyzing or visualizing feature distribution
xdata_imputed.hist(bins=30,figsize=(25,15))

In [ ]:


sns.boxplot(data=xdata_imputed)
plt.title('Box Plot for Outlier Detection')
plt.show()


In [ ]:
#checkig the outliers
from algorithm_scripts.outlier_detection import IQR

outliers = IQR(xdata_imputed)
outliers.sum().sum()

In [ ]:

from algorithm_scripts.outlier_detection import iqrImpute
imputed_data = iqrImpute(xdata_imputed)
imputed_data.describe()


In [ ]:


sns.boxplot(data=imputed_data)
plt.title('Box Plot for Outlier Detection')
plt.show()

In [ ]:
xdata_imputed = imputed_data
xdata_imputed.head()


In [ ]:
#doing correlation analysis 
corr_marix = xdata_imputed.corr()
plt.figure(figsize=(25,15))
sns.heatmap(corr_marix,annot=True,cmap='coolwarm')

In [50]:
# #now checking for balancing or imbalancinfg the dataset
class_distribution = y.value_counts()
class_distribution
from algorithm_scripts.balancing_dataset import smote

balanced_data = smote(xdata_imputed,y)
x_balanced = balanced_data[0]
y_balanced = balanced_data[1]



In [51]:
xdata_imputed = iqrImpute(x_balanced)
y = y_balanced


In [ ]:
print(xdata_imputed.shape)
print(y.shape)

In [ ]:
from models.svm import train_svm_model

accuracy = train_svm_model(xdata_imputed,y=y)
accuracy